# Empower Sequence Labeling with Task-Aware Language Model

Setup system paths and modules:

In [1]:
import sys, os

ditk_path = os.path.abspath(os.path.join("../../.."))
if ditk_path not in sys.path:
    sys.path.append(ditk_path)


## Importing modules

In [2]:
from extraction.named_entity.lmlstmcrf.hparams import hparams as hp
from extraction.named_entity.lmlstmcrf.lmlstmcrf import Lmlstmcrf

## Setting up file paths

In [3]:
data_dir = os.path.join(ditk_path, 'extraction/named_entity/lmlstmcrf')

data_sets = ['conll', 'ontonotes', 'chemdner']

emb_file = data_dir + "/embedding/glove.6B.100d.embedding"

# choose dataset to use here
chosen_type = data_sets[0]
train_file = data_dir + "/data/{}/out.train".format(chosen_type)
dev_file = data_dir + "/data/{}/out.testa".format(chosen_type)
test_file = data_dir + "/data/{}/out.testb".format(chosen_type)

# set hparam files
hp.checkpoint_dir = data_dir + "/checkpoint/"
hp.checkpoint = data_dir + "/checkpoint/conll.model"
hp.emb_file = emb_file

if not os.path.exists(hp.checkpoint_dir):
    os.makedirs(hp.checkpoint_dir)
    
# set gpu
hp.gpu = -1 # -1 for no gpu

## Creating LMLSTMCRF object

In [4]:
model = Lmlstmcrf()

setting:
<class 'extraction.named_entity.lmlstmcrf.hparams.hparams'>


## Read dataset

In [5]:
data = model.read_dataset({
    "train": train_file,
    "test": test_file,
    "dev": dev_file,
})

## Load pretrained model

In [6]:
model.load_model(hp.checkpoint)

/anaconda3/envs/lmlstmcrf/lib/python3.7/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.55 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


## Train model

In [9]:
model.train(data) # takes a very long time without gpu

## Make prediction

In [7]:
predictions = model.predict(data)

annotating
 - Process:   0%|          | 0/3453 [00:00<?, ?it/s]

/anaconda3/envs/lmlstmcrf/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [8]:
# output example 
predictions[:10]

[('SOCCER', 'O', 'O'),
 ('-', 'O', 'O'),
 ('Nadim', 'I-PER', 'I-PER'),
 ('Ladki', 'I-PER', 'I-PER'),
 ('AL-AIN', 'I-ORG', 'I-LOC'),
 (',', 'I-ORG', 'O'),
 ('Japan', 'I-LOC', 'I-LOC'),
 ('began', 'O', 'O'),
 ('But', 'O', 'O'),
 ('China', 'O', 'I-LOC')]

## Evaluate model

In [9]:
score = model.evaluate(predictions, data)
print(score)

start
/Users/kevinli/school/csci548/project/ditk/extraction/named_entity/lmlstmcrf/checkpoint/conll.model : total : test_f1: 0.9078 test_rec: 0.9069 test_pre: 0.9088 test_acc: 0.9813 | 

/Users/kevinli/school/csci548/project/ditk/extraction/named_entity/lmlstmcrf/checkpoint/conll.model : I-PER : test_f1: 0.9708 test_rec: 0.9715 test_pre: 0.9701 test_acc: 0.0000 | {'I-ORG': 30, 'O': 19, 'I-LOC': 30}

/Users/kevinli/school/csci548/project/ditk/extraction/named_entity/lmlstmcrf/checkpoint/conll.model : I-LOC : test_f1: 0.9234 test_rec: 0.9479 test_pre: 0.9000 test_acc: 0.0000 | {'I-MISC': 29, 'I-ORG': 53, 'I-PER': 10, 'O': 8}

/Users/kevinli/school/csci548/project/ditk/extraction/named_entity/lmlstmcrf/checkpoint/conll.model : O : test_f1: 0.9946 test_rec: 0.9937 test_pre: 0.9955 test_acc: 0.0000 | {'I-MISC': 113, 'I-ORG': 74, 'I-LOC': 37, 'I-PER': 19}

/Users/kevinli/school/csci548/project/ditk/extraction/named_entity/lmlstmcrf/checkpoint/conll.model : I-MISC : test_f1: 0.8100 test_rec: 